In [1]:
import requests
import numpy as np


FORECAST_URL = 'https://api.open-meteo.com/v1/forecast'

HISTORICAL_DATA_URL = 'https://archive-api.open-meteo.com/v1/era5'

FORECAST_PARAMS = {
    'latitude':39.9439,
    'longitude': 32.8560,
    'hourly': ['temperature_2m', 'relativehumidity_2m', 'dewpoint_2m', 'precipitation', 'rain', 'windspeed_10m', 'winddirection_10m', 'soil_moisture_9_27cm', 'cloudcover']
}

HISTORICAL_PARAMS = {
    'latitude':39.9439,
    'longitude': 32.8560,
    'hourly': ['temperature_2m', 'relativehumidity_2m', 'dewpoint_2m', 'precipitation', 'rain', 'windspeed_10m', 'soil_moisture_7_to_28cm', 'cloudcover']
}

forecast_r = requests.get(url = FORECAST_URL, params = FORECAST_PARAMS)

HISTORICAL_PARAMS['start_date'] = '2021-01-01'
HISTORICAL_PARAMS['end_date'] = '2022-01-01'

historical_r = requests.get(url = HISTORICAL_DATA_URL, params = HISTORICAL_PARAMS)

forecast_data = forecast_r.json()
historical_data = historical_r.json()



In [2]:
for i in forecast_data['hourly']:
    print(i, end=' ')
print('****************')
for i in historical_data['hourly']:
    print(i, end=' ')

# print('****************')
# print('Forecast Data for 2 meters temperature')

# for element in zip(forecast_data['hourly']['time'][:100], forecast_data['hourly']['temperature_2m'][:100], forecast_data['hourly']['relativehumidity_2m'][:100]
# , forecast_data['hourly']['rain'][:100]):
#     print(element)
    
# print('****************')
# print('Historical Data for 2 meters temperature')    

for i, element in enumerate(zip(historical_data['hourly']['time'][:10], historical_data['hourly']['temperature_2m'][:10])):
    print(i, element)

print()
print(len(forecast_data['hourly']['temperature_2m']))
print(len(historical_data['hourly']['temperature_2m'])/24)

time temperature_2m relativehumidity_2m dewpoint_2m precipitation rain windspeed_10m winddirection_10m soil_moisture_9_27cm cloudcover ****************
time temperature_2m relativehumidity_2m dewpoint_2m precipitation rain windspeed_10m soil_moisture_7_to_28cm cloudcover 0 ('2021-01-01T00:00', 2.1)
1 ('2021-01-01T01:00', 2.1)
2 ('2021-01-01T02:00', 1.6)
3 ('2021-01-01T03:00', 1.1)
4 ('2021-01-01T04:00', 0.6)
5 ('2021-01-01T05:00', 0.1)
6 ('2021-01-01T06:00', 1.5)
7 ('2021-01-01T07:00', 3.5)
8 ('2021-01-01T08:00', 7.6)
9 ('2021-01-01T09:00', 9.9)

168
366.0


In [3]:
def get_pth_percentile(array, percentile):
    sorted = np.sort(array)

    ith = int((percentile/100)*(len(array)-1))

    return sorted[ith]


In [4]:
high_thresholds = {}
for i in historical_data['hourly']:
    high_thresholds[i] = get_pth_percentile(historical_data['hourly'][i], 90)

low_thresholds = {}
for i in historical_data['hourly']:
    low_thresholds[i] = get_pth_percentile(historical_data['hourly'][i], 10)

print(high_thresholds)
print(low_thresholds)

{'time': '2021-11-26T08:00', 'temperature_2m': 26.3, 'relativehumidity_2m': 88, 'dewpoint_2m': 11.6, 'precipitation': 0.0, 'rain': 0.0, 'windspeed_10m': 13.2, 'soil_moisture_7_to_28cm': 0.291, 'cloudcover': 96}
{'time': '2021-02-06T14:00', 'temperature_2m': 0.7, 'relativehumidity_2m': 28, 'dewpoint_2m': -4.0, 'precipitation': 0.0, 'rain': 0.0, 'windspeed_10m': 3.4, 'soil_moisture_7_to_28cm': 0.108, 'cloudcover': 0}


### Increasing Irrigation

Temperature is high / humidity is low / wind speed is high

In [7]:
def increase_irrigation_alert(forecasted_data):
    alerts = {}
    day = 0
    for i, row in enumerate(zip(forecasted_data['temperature_2m'], forecasted_data['relativehumidity_2m'], forecasted_data['windspeed_10m'])):
        if(i%24 == 0):
            day += 1
            alerts[day] = []
        #     print(f'DAY {day}: ')
        # print(i, row)
        if(row[0] > high_thresholds['temperature_2m']):
            alerts[day].append(('temperature_2m', forecasted_data['time'][i]))

        if(row[1] < low_thresholds['relativehumidity_2m']):
            alerts[day].append(('relativehumidity_2m', forecasted_data['time'][i]))

        if(row[2] > high_thresholds['windspeed_10m']):
            alerts[day].append(('windspeed_10m', forecasted_data['time'][i]))

    return alerts

increase_irrigation_alert(forecast_data['hourly'])


{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: []}

### Decreasing Irrigation

Rain, precipitation is high / soil moisture is high

In [38]:
def decrease_irrigation_alert(forecasted_data):
    alerts = {}
    day = 0
    for i, row in enumerate(zip(forecasted_data['precipitation'], forecasted_data['soil_moisture_9_27cm'])):
        if(i%24 == 0):
            day += 1
            alerts[day] = {}
            alerts[day]['precipitation'] = []
            alerts[day]['soil_moisture_9_27cm'] = []
        #     print(f'DAY {day}: ')
        # print(i, row)
        if(row[0] > high_thresholds['precipitation']):
            alerts[day]['precipitation'].append(forecasted_data['time'][i])

        if(row[1] > high_thresholds['soil_moisture_7_to_28cm']):
            alerts[day]['soil_moisture_9_27cm'].append(forecasted_data['time'][i])
    return alerts

x = decrease_irrigation_alert(forecast_data['hourly'])


### Frost Warning

- A forecasted overnight low less than 40 degrees F. (4.4 C.). 
- A dewpoint lower than 40 degrees F. 
- A cloudless night. (Be aware skies can clear overnight.) 
- A windless night.

Read more at Gardening Know How: How To Predict Frost And Protect Your Garden https://www.gardeningknowhow.com/plant-problems/environmental/how-to-predict-frost.htm

### Recommendation of Irrigation Schedule

May be determined according to the sunrise/sunset

## Recommending Based On Alert Messages

In [52]:
week_dict = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}

def hour(time_string):
    return int(time_string[-5:-3])

def investigate_one_day(day_message):
    if(len(day_message) == 0): return []
    message_intervals = []

    i = 0
    current_first_end = day_message[i]
    while(True):
        current_last_end = day_message[i]
        if(i >= len(day_message)-1):
            message_intervals.append((current_first_end, current_last_end))
            break
        if(hour(day_message[i]) + 1 != hour(day_message[i+1])):
            message_intervals.append((current_first_end, current_last_end))
            current_first_end = day_message[i+1]
        i += 1

    return message_intervals

def recommend(alert_message, today = 1):
    for i in range(1, 8):
        current_day = week_dict[(today+i-1)%7+1]
        print(f'\nNotifications for {current_day}:')
        for type in alert_message[i]:
            if(alert_message[i][type] != []):
                print(f'In the following hours the {type} is high:')
                intervals = investigate_one_day(alert_message[i][type])
                for interval in intervals:
                    print(f'between {interval[0]} - {interval[1]}')
                print('You may consider decreasing irrigation')

    pass

recommend(x)


Notifications for Tuesday:
In the following hours the soil_moisture_9_27cm is high:
between 2022-12-27T00:00 - 2022-12-27T23:00
You may consider decreasing irrigation

Notifications for Wednesday:
In the following hours the soil_moisture_9_27cm is high:
between 2022-12-28T00:00 - 2022-12-28T23:00
You may consider decreasing irrigation

Notifications for Thursday:
In the following hours the soil_moisture_9_27cm is high:
between 2022-12-29T00:00 - 2022-12-29T23:00
You may consider decreasing irrigation

Notifications for Friday:
In the following hours the soil_moisture_9_27cm is high:
between 2022-12-30T00:00 - 2022-12-30T23:00
You may consider decreasing irrigation

Notifications for Saturday:
In the following hours the soil_moisture_9_27cm is high:
between 2022-12-31T00:00 - 2022-12-31T10:00
You may consider decreasing irrigation

Notifications for Sunday:

Notifications for Monday:
